In [2]:
import torch
import pandas as pd
import numpy as np
import torchio as tio
from torch.utils.data import DataLoader
import os
import SimpleITK as sitk
from zipfile import ZipFile
from zipfile import BadZipFile
import dask.dataframe as dd
import os
import multiprocessing as mp
import functools
from functools import partial
import Standardize


## managment of files 
managment of files is done via managePicaiFiles.sh - create directories download and unpack files saves basic metadata and do simple metadata preprocessing
sh managePicaiFiles.sh

## Standarization
primary preprocessing - removing ouliers, put values between 0 and 255 bias field correction Nyul standarization binarizing labels

In [2]:
df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
df['sag']

0       /home/sliceruser/data/orig/10000/10000_1000000...
1       /home/sliceruser/data/orig/10001/10001_1000001...
2       /home/sliceruser/data/orig/10002/10002_1000002...
3       /home/sliceruser/data/orig/10003/10003_1000003...
4       /home/sliceruser/data/orig/10004/10004_1000004...
                              ...                        
1495    /home/sliceruser/data/orig/11471/11471_1001495...
1496    /home/sliceruser/data/orig/11472/11472_1001496...
1497    /home/sliceruser/data/orig/11473/11473_1001497...
1498    /home/sliceruser/data/orig/11474/11474_1001498...
1499    /home/sliceruser/data/orig/11475/11475_1001499...
Name: sag, Length: 1500, dtype: object

In [1]:
import Standardize
import pandas as pd

df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
numRows=3
Standardize.iterateAndchangeLabelToOnes(numRows,df)
for keyWord in ['t2w','adc', 'cor','hbv','sag'  ]:
    Standardize.iterateAndStandardize(keyWord,numRows,df)

df.to_csv('/home/sliceruser/data/metadata/processedMetaData.csv') 


newwD
newwD
newwD
biasFieldCorrect /home/sliceruser/data/orig/10001/10001_1000001_t2w.mha
biasFieldCorrect /home/sliceruser/data/orig/10002/10002_1000002_t2w.mha
biasFieldCorrect /home/sliceruser/data/orig/10000/10000_1000000_t2w.mha
fitting normalizer


/home/sliceruser/Slicer/lib/Python/lib/python3.9/site-packages/intensity_normalization/normalize/base.py:89: UserWarning: Data contains negative values; skull-stripped functionality assumes the foreground is all positive. Provide the brain mask if otherwise.
  warnings.warn(msg)


standardizeFromPathAndOverwrite /home/sliceruser/data/orig/10001/10001_1000001_t2w.mha
standardizeFromPathAndOverwrite /home/sliceruser/data/orig/10000/10000_1000000_t2w.mha
standardizeFromPathAndOverwrite /home/sliceruser/data/orig/10002/10002_1000002_t2w.mha
newwD
newwD
newwD
biasFieldCorrect /home/sliceruser/data/orig/10002/10002_1000002_adc.mha
biasFieldCorrect /home/sliceruser/data/orig/10000/10000_1000000_adc.mha
biasFieldCorrect /home/sliceruser/data/orig/10001/10001_1000001_adc.mha
fitting normalizer
standardizeFromPathAndOverwrite /home/sliceruser/data/orig/10000/10000_1000000_adc.mha
standardizeFromPathAndOverwrite /home/sliceruser/data/orig/10001/10001_1000001_adc.mha
standardizeFromPathAndOverwrite /home/sliceruser/data/orig/10002/10002_1000002_adc.mha
newwD
newwD
newwD
biasFieldCorrect /home/sliceruser/data/orig/10002/10002_1000002_cor.mha
biasFieldCorrect /home/sliceruser/data/orig/10001/10001_1000001_cor.mha
biasFieldCorrect /home/sliceruser/data/orig/10000/10000_1000000

## Trying Unires

In [ ]:
#Frst we need to save files as nifti- we will keep the same folder structure just alternate file type
def saveAsNifti(path):
    image=sitk.ReadImage(path)
    newPath = path.replace(".mha",".nii.gz")
    writer = sitk.ImageFileWriter()
    writer.KeepOriginalImageUIDOn()
    writer.SetFileName(newPath)
    writer.Execute(image)
    return newPath

for keyWord in ['t2w','adc', 'cor','hbv','sag','reSampledPath']:
    train_patientsPaths=df[keyWord].dropna().astype('str').to_numpy()
    train_patientsPaths=list(filter(lambda path: len(path)>2 ,train_patientsPaths))
    with mp.Pool(processes = mp.cpu_count()) as pool:
        pool.map(saveAsNifti,train_patientsPaths)



### playground

In [1]:
from unires.struct import settings as s
from unires.run import preproc


import SimpleITK as sitk
import pandas as pd

df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')
numb= 19

pathT2w=df['t2w'].to_numpy()[numb]
pathCor=df['cor'].to_numpy()[numb]
pathSag=df['sag'].to_numpy()[numb]
pathHbv=df['hbv'].to_numpy()[numb]
pathAdc=df['adc'].to_numpy()[numb]
pathLabel=df['reSampledPath'].to_numpy()[numb]




def saveAsNifti(path):
    image=sitk.ReadImage(path)
    newPath = path.replace(".mha",".nii.gz")
    writer = sitk.ImageFileWriter()
    writer.KeepOriginalImageUIDOn()
    writer.SetFileName(newPath)
    writer.Execute(image)
    return newPath
pathT2wNew = saveAsNifti(pathT2w)
pathCorNew = saveAsNifti(pathCor)
pathSagNew=saveAsNifti(pathSag)
pathHbvNew=saveAsNifti(pathHbv)
pathAdcNew=saveAsNifti(pathAdc)
pathLabelNew=saveAsNifti(pathLabel)


df['isAnythingInAnnotated'].to_numpy()[numb]

#image1=sitk.ReadImage(pathT2w)
# pathT2w.replace(".mha",".nii.gz")

262.0

In [4]:
print(pathT2wNew)
print(pathCorNew)
print(pathSagNew)
sett=s()
sett.device='cpu' #TODO() remove
#sett.do_atlas_align='false'
sett.do_atlas_align=False
#sett.cgs_max_iter=3#TODO() remove
sett.max_iter=110#TODO() remove
sett.dir_out = '/home/sliceruser/data/Unires'
sett.label= pathLabelNew


# in command line
# unires /home/sliceruser/data/orig/10000/10000_1000000_t2w.nii.gz /home/sliceruser/data/orig/10000/10000_1000000_cor.nii.gz /home/sliceruser/data/orig/10000/10000_1000000_sag.nii.gz


/home/sliceruser/data/orig/10019/10019_1000019_t2w.nii.gz
/home/sliceruser/data/orig/10019/10019_1000019_cor.nii.gz
/home/sliceruser/data/orig/10019/10019_1000019_sag.nii.gz


In [ ]:
res = preproc([pathT2wNew, pathCorNew,pathSagNew,pathHbvNew,pathAdcNew ],sett=sett)


In [12]:
a='/home/sliceruser/data/Unires/ur_10000_1000000_cor.nii.gz'
b='/home/sliceruser/data/Unires/ur_10000_1000000_sag.nii.gz'
c='/home/sliceruser/data/Unires/ur_10000_1000000_t2w.nii.gz'
image=sitk.ReadImage(c)
image.GetSpacing()

(1.0, 1.0, 1.0)

In [ ]:
(203, 251, 252)

## Getting all of the spacings 

In [3]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# if using a Jupyter notebook, includue:
%matplotlib inline


In [9]:
df = pd.read_csv('/home/sliceruser/data/metadata/processedMetaData.csv')

"""
looking through all valid spacings (if it si invalid it goes below 0)
and displaying minimal maximal and rounded mean spacing and median
in my case median and mean values are close - and using the median values will lead to a bit less interpolations later
"""
spacingDict={}
for keyWord in ['t2w','adc', 'cor','hbv','sag'  ]: 
    for addedKey in ['_spac_x','_spac_y','_spac_z']:   
        colName = keyWord+addedKey
        liist = list(filter(lambda it: it>0 ,df[colName].to_numpy() ))
        minn=np.min(liist)                
        maxx=np.max(liist)
        meanRounded = round((minn+maxx)/2,1)
        medianRounded = round(np.median(liist),1)
        spacingDict[colName]=(minn,maxx,meanRounded,medianRounded)

spacingDict



{'t2w_spac_x': (0.234375, 0.78125, 0.5, 0.5),
 't2w_spac_y': (0.234375, 0.78125, 0.5, 0.5),
 't2w_spac_z': (2.200000060372773, 5.000000066297424, 3.6, 3.0),
 'adc_spac_x': (0.859375, 2.5098040103912, 1.7, 2.0),
 'adc_spac_y': (0.859375, 2.5098040103912, 1.7, 2.0),
 'adc_spac_z': (2.999998832062665, 5.000000232855622, 4.0, 3.0),
 'cor_spac_x': (0.286756128073, 0.9765625, 0.6, 0.6),
 'cor_spac_y': (0.286756128073, 0.9765625, 0.6, 0.6),
 'cor_spac_z': (0.9999999999999636, 4.49999998188102, 2.7, 3.0),
 'hbv_spac_x': (0.859375, 2.5098040103912, 1.7, 2.0),
 'hbv_spac_y': (0.859375, 2.5098040103912, 1.7, 2.0),
 'hbv_spac_z': (2.999998832062665, 5.000000232855622, 4.0, 3.0),
 'sag_spac_x': (0.254629641771, 0.78125, 0.5, 0.6),
 'sag_spac_y': (0.254629641771, 0.78125, 0.5, 0.6),
 'sag_spac_z': (2.999999882111563, 4.000404550206146, 3.5, 3.6)}

Now idea is to iteratively transform images into uniform spacing here i will choose median of the data as the target value

In [10]:
from KevinSR import mask_interpolation, SOUP_GAN
import os
import numpy as np
from scipy import ndimage, interpolate
from scipy.ndimage import zoom
from KevinSR import SOUP_GAN
import monai
import SimpleITK as sitk
from copy import deepcopy


def superSampleinZ(image, currentSpacZ, targetSpacZ):
    """
    perform supersampling in Z direction
    currentSpac, targetSpac - current and target z spacing - float numbers
    """
    pass


def subsample(image,currentSpac, targetSpac):
    """
    perform subsampling to targetSpac
    """    
    pass


def transformSpacingTo(path, targetSpac):
    """
    transforming spacing of the image which path is supplied to target spacing values
    upsampling will be performed with SOUP-GAN  and downsampling will be done via simple itk bicubic interpolation
    """
    image = sitk.ReadImage(path)
    spac = image.GetSpacing()
    for axis in [0,1,2]:
        if(targetSpac[axis]>spac[axis]):
            #if target spacing is bigger we need to subsample
            spacGoal=deepcopy(spac)
            spacGoal[axis]=
            subsample(image, )
    


In [7]:
# from KevinSR import mask_interpolation, SOUP_GAN
# import os
# import numpy as np
# from scipy import ndimage, interpolate
# from scipy.ndimage import zoom
# from KevinSR import SOUP_GAN
# import monai

# import SimpleITK as sitk

# path = df['t2w'].to_numpy()[0]



# spacingX_a= 0.234375
# spacingX_b= 0.78125

# spacingY_a= 0.234375
# spacingY_b= 0.78125

# spacingZ_a= 2.200000060372773
# spacingZ_b= 5.000000066297424

# img = sitk.ReadImage(path)
# image1 = sitk.ReadImage(path)

# thicks_ori = sitk.GetArrayFromImage(image1)
# Z_FAC = spacingZ_b/spacingZ_a

# # Call the SR interpolation tool from KevinSR
# thins_gen = SOUP_GAN(thicks_ori, Z_FAC, 0)




# #recreating image keeping relevant metadata
# image = sitk.GetImageFromArray(data)
# image.SetSpacing(image1.GetSpacing())
# image.SetOrigin(image1.GetOrigin())
# image.SetDirection(image1.GetDirection())



# #Example of thick-to-thin type of preprocessing
# pre_slices = 27
# post_slices = 135


# # thicks_ori = load_data(DIR_dicom)
# # Z_FAC = post_slices/pre_slices # Sampling factor in Z direction

# # thicks_ori = rescale_img(thicks_ori, max_val= 10000)

# # thins = zoom(thicks_ori, (1,1,Z_FAC))
# # thins_raw = zoom(thicks_ori, (1,1,Z_FAC),order=0)

# # # Call the SR interpolation tool from KevinSR
# # thins_gen = SOUP_GAN(thicks_ori, Z_FAC, 0)

# # # Plot the original thick slices, thin slices by TCI and SR generated slices. 
# # plot_scans(thins_raw, thins, thins_gen, 0)


In [8]:
# for keyWord in ['t2w','adc', 'cor','hbv','sag'  ]: 
#     for addedKey in ['_sizz_x','_sizz_y','_sizz_z','_spac_x'
#                         ,'_spac_y','_spac_z'
#                         ,'_orig_x','_orig_x','_orig_x']:  

## groupwise registration

In [ ]:
https://github.com/SuperElastix/SimpleElastix/issues/255
with ants 
https://github.com/SunYH66/adn-master/blob/d69a73e2f9cf2a4472c1d97f7347677b1947543a/adn/utils/preprocessing.py
